In [1]:
# ThermostatSC2

In [2]:
import datetime
import logging
import random
import time
import uuid

import smartpynector as sp
from utils import *

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ThermostatSC")

In [3]:
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
}

TEMP_MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?temp .
                        ?meas saref:isMeasuredIn saref:TemperatureUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Temperature .
                        ?meas saref:measurementMadeBy ?device_id ."""

TEMP_SETTING_GRAPH_PATTERN = """?setting rdf:type saref:SetLevelCommand .
                            ?setting saref:hasValue ?desired_temp .
                            ?setting saref:isMeasuredIn saref:TemperatureUnit .
                            ?setting saref:hasTimestamp ?timestamp .
                            ?setting saref:isCommandOf ?room_id .
                            ?setting saref:relatesToProperty saref:Temperature .
                            ?setting saref:commandIssuedBy ?device_id .
                            """

HUMIDITY_MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?humidity .
                        ?meas saref:isMeasuredIn saref:HumidityUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Humidity .
                        ?meas saref:measurementMadeBy ?device_id ."""

HUMIDITY_SETTING_GRAPH_PATTERN = """?setting rdf:type saref:SetLevelCommand .
                            ?setting saref:hasValue ?desired_humidity .
                            ?setting saref:isMeasuredIn saref:HumidityUnit .
                            ?setting saref:hasTimestamp ?timestamp .
                            ?setting saref:isCommandOf ?room_id .
                            ?setting saref:relatesToProperty saref:Humidity .
                            ?setting saref:commandIssuedBy ?device_id .
                            """

In [4]:
def get_temp_now():
    return requests.get(THERMOSTAT_API_URL + "/actual_temperature").json()

In [5]:
def handle_react_change_desired_temp(bindings):
    for binding in bindings:
        logger.info(f"Changing desired temperature to: {binding['desired_temp']}")
        # change desired temp
        requests.put(
            THERMOSTAT_API_URL
            + "/desired_temperature"
            + f'?desired_temperature={int(binding["desired_temp"])}'
        )
    return []

In [6]:
import threading


def start_sensor_kb(
    kb_id, kb_name, kb_description, ke_endpoint, post_live_measurements=True
):
    # delete in case allready exists
    delete_knowledge_base(kb_id, ke_endpoint)

    # register kb
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    post_ki_id = register_post_knowledge_interaction(
        TEMP_MEAS_GRAPH_PATTERN,
        None,
        "post-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    react_ki_id = register_react_knowledge_interaction(
        TEMP_SETTING_GRAPH_PATTERN,
        None,
        "set-desired-temp",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    def measurement_loop(post_ki_id, kb_id, ke_endpoint):
        while True:
            value = get_temp_now()

            post(
                [
                    {
                        "meas": f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>",
                        "temp": f"{get_temp_now()}",
                        "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
                        "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
                        "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
                    }
                ],
                post_ki_id,
                kb_id,
                ke_endpoint,
            )
            logger.info(
                f"published measurement of {value} units at {sp.get_timestamp_now()}"
            )

            time.sleep(5)

    # Start the measurement_loop function in a separate thread
    measurement_thread = threading.Thread(
        target=measurement_loop,
        args=(
            post_ki_id,
            kb_id,
            ke_endpoint,
        ),
    )

    # Post live measurements if post_live_measurements is set to True
    if post_live_measurements:
        measurement_thread.start()

    # Start the start_handle_loop function in a separate thread
    start_handle_loop(
        {
            react_ki_id: handle_react_change_desired_temp,
        },
        kb_id,
        ke_endpoint,
    )

In [7]:
start_sensor_kb(
    "http://example.org/sensor",
    "Thermostat1_temp",
    "A thermostat",
    "http://knowledge_engine:8280/rest/",
)

2023-06-22 08:18:09 INFO {"messageType":"error","message":"Deletion of knowledge base failed, because it could not be found."}
2023-06-22 08:18:10 INFO registered Thermostat1_temp
2023-06-22 08:18:11 INFO received issued knowledge interaction id: http://example.org/sensor/interaction/post-measurements
2023-06-22 08:18:11 INFO received issued knowledge interaction id: http://example.org/sensor/interaction/set-desired-temp
2023-06-22 08:18:11 INFO published measurement of 22 units at 2023-06-22T08:18:11+00:00
2023-06-22 08:18:16 INFO published measurement of 18 units at 2023-06-22T08:18:16+00:00
2023-06-22 08:18:21 INFO published measurement of 19 units at 2023-06-22T08:18:21+00:00
2023-06-22 08:18:26 INFO published measurement of 19 units at 2023-06-22T08:18:26+00:00
2023-06-22 08:18:32 INFO published measurement of 18 units at 2023-06-22T08:18:32+00:00
2023-06-22 08:18:37 INFO published measurement of 22 units at 2023-06-22T08:18:37+00:00
2023-06-22 08:18:43 INFO published measurement 

KeyboardInterrupt: 